In [7]:
import re
from time import time
import numpy as np
from operator import add

data = sc.wholeTextFiles('dataset/reut2-*')
newsArticles = data.map(lambda x:x[1]).flatMap(lambda x:x.split('<BODY>')[1:]).map(lambda x:x.split('</BODY>')[0])\
                   .map(lambda x:re.sub(' +', ' ', x.replace('\n', ' ')))
newsArticles.take(3)

["Gold share prices closed mixed to slightly firmer in quiet and cautious trading, showing little reaction to a retreat in the bullion price back to below 420 dlrs and a firmer financial rand, dealers said. Heavyweight Vaal Reefs ended eight rand higher at 398 rand but Grootvlei eased 40 cents at 16.60 rand, while mining financials had Gold Fields up a rand at 63 rand despite weaker quarterly results. Other minings were firm but platinums eased. Industrials also closed mixed to firmer, the index once again hitting a new high of 1757 from Friday's 1753 finish. The overall index also hit a new high of 2188 versus 2179 on Friday. REUTER &#3;",
 "Feeder Cattle futures advanced 0.20 to 0.30 cent at the start and posted new season's highs in April through August and October before slipping to trade 0.10 cent lower to 0.20 higher in latest trade. Futures ran up to new highs at the start on general demand prompted by continued strong cattle fundamentals. Posting of another 10 cent gain in the 

### Q#1. Given the Reuters-21578 dataset, please calculate all k-shingles and output the set representation of the text dataset as a matrix.

In [8]:
#Enter the K value of K-Shingles from the user
k=int(input("Please Enter the K value of K-Shingles:- "))

Please Enter the K value of K-Shingles:- 3


In [9]:
shingles = newsArticles.flatMap(lambda x:[x[i:i+k] for i in range(len(x)-k+1)]).distinct()
shingles_count = shingles.count()
articles_count = newsArticles.count()
print('different shingles-->',shingles_count)
print('different articles-->',articles_count)

different shingles--> 35077
different articles--> 19043


In [10]:
newsArticles = newsArticles.collect()
k_shingles_matrix = shingles.map(lambda s:[1 if s in a else 0 for a in newsArticles])
k_shingles_matrix.coalesce(1).saveAsTextFile('tmp_hw3')

'''
move the file from tmp_hw3 folder to hw_3_result folder and rename the file finally delete the tmp_hw3 folder
'''
!mv tmp_hw3/part-00000 hw_3_result/k_shingles_matrix.txt
!rm -rf tmp_hw3

### Q2 Given the set representation, compute the minhash signatures of all documents using MapReduce.

In [11]:
def biggerThanNFirstPrime(N):
    p = 2
    while True:
        isPrime = True
        for i in range(2,p//2+1):
            if(p%i==0):
                isPrime = False
                break
        if isPrime and p > N:
            return p
        else:
            p+=1

In [12]:
#Take the number of randum hash functions from the user
h=int(input("Please Enter the number of hash functions:- "))

Please Enter the number of hash functions:- 100


In [18]:
import random
a = [random.randint(0, h) for i in range(h)]
b = [random.randint(0, h) for i in range(h)]
p = biggerThanNFirstPrime(articles_count)
N = articles_count

def rowHash(row, a, b, p, N):
    return ((a*row+b)%p)%N

In [23]:
from time import clock
minHashSignatures = list()
#intialize time
initial_time=time()
kShinglesMatrixZipWithIndex = k_shingles_matrix.zipWithIndex().cache()
for i in range(h):
    minHashSignatures.append(kShinglesMatrixZipWithIndex\
                             .map(lambda x:[rowHash(x[1], a[i], b[i], p ,N) if c == 1 else (articles_count + 10) for c in x[0]])\
                             .reduce(lambda x, y:[Mx if Mx < My else My for Mx, My in zip(x, y)]))

In [ ]:
with open('hw_3_result/min_hash_signatures.txt', 'w') as result:
    for row in minHashSignatures:
        result.write(str(row) + '\n') 

#### Q#3 Implement the LSH algorithm by MapReduce and output the resulting candidate pairs of similar documents.

In [37]:
#please enter the band size from the user sample value 20
bands=int(input("Please Enter the band size:- "))

Please Enter the band size:- 20


In [ ]:
r = 5
similarRate = 0.8
buckets = articles_count
hashFuct = [[random.randint(0, 100) for i in range(r + 1)] for j in range(bands)]

with open('hw_3_result/candidate_pairs.txt', 'w') as result:
    for i in range(articles_count):
        candidatePairs = list()
        for j in range(bands):
            band = np.array(minHashSignatures[j*r:j*r+r]).T
            band = [(np.array(article).dot(np.array(hashFuct[j][:r])) + hashFuct[j][-1]) % buckets for article in band]
            for k, article in enumerate(band):
                if k > i and article == band[i]:
                    candidatePairs.append(k)
        candidatePairs = [(article, candidatePairs.count(article)) for article in set(candidatePairs)]
        candidatePairsTreshold = list()
        for candidatePair in candidatePairs:
            if candidatePair[1] >= bands*similarRate:
                candidatePairsTreshold.append(candidatePair[0])
        result.write('Articles' + str(i) + ':' + str(candidatePairsTreshold) + '\n')